## One Jupyter notebook or many?

**Advantages** of one notebook:
* Students don't get lost looking for the current notebook file.
* Variables and functions can be reused without needing to be redefined.

**Disadvantages** of one notebook:
* Students can get lost scrolling up and down the file.
* Variables and functions might get redefined with subtly different meanings through a notebook. If they evaluate it out of order, it won't work correctly.

<br><br><br>

## Should students run the same notebook you're running?

It may seem obvious that they should, but suppose you have a lot of expository material sprinkled with small exercises. Usually, the code for the exercises won't work unless the students have been executing all the cells along with you—hitting "shift-enter" through the notebook.

If they haven't been hitting "shift-enter", they have to scramble to figure out how much of the notebook to run to get to the exercise in the short amount of time you give them to do it.

If they have been hitting "shift-enter", their attention has been divided between the big screen you're presenting from and the little screen in front of them. That's a lot of context switching!

My suggestion for short exercises: separate your `lecture.ipynb` from their `lecture-workbook.ipynb` and fill the latter with only as much code as is needed for the short exercises.

**Example:** [jpivarski-talks/2025-07-07-scipy2025-tutorial-thinking-in-arrays](https://github.com/jpivarski-talks/2025-07-07-scipy2025-tutorial-thinking-in-arrays)

<br><br><br>

Alternatively, don't do short exercises at all! You always lose time to context-switching, and the time may be better spent in a long exercise of a half hour or more.

<br><br><br>

## Hiding hints and solutions

HTML has a lot of useful tricks. Consider `<details>`:

<details>
    <summary><b>Hint:</b></summary>

You can have markdown, _like this_, inside of HTML in a markdown cell.

</details>

<br><br><br>

## More HTML goodies

Markdown cells don't support very complex HTML, but code cells starting with `%%html` do.

This one lets you embed another website in your notebook, which has various uses.

In [ ]:
%%html

<div style="overflow: hidden;">
    <iframe src="https://tryapl.org/" width="100%" height="380" scrolling="no" style="border: none;">
</div>

<br><br><br>

If you need to generate HTML (or audio, images, etc.) with Python, you can do that with [IPython.display](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html).

In [ ]:
from IPython.display import display, HTML

In [ ]:
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
n = int(input("n ="))

display(HTML(f"""
<div style="font-size: 32px; margin: 100px;">
    The <b>{n}th</b> Fibonacci number is <b>{fibonacci(n)}</b>.
</div>
"""))

Or use [`_repr_html_`, `_repr_svg_`, etc.](https://ipython.readthedocs.io/en/stable/config/integrating.html) to make Python objects represent themselves as HTML, images, etc.

(This is how Pandas presents DataFrames as neat tables.)

In [ ]:
# A big code block that should be in a mini-library!

import xml.etree.ElementTree

class SVG(xml.etree.ElementTree.Element):
    def __str__(self):
        return xml.etree.ElementTree.tostring(svg.svg(
            self._builder.__getattr__("defs")(*self._builder.defs), self, **self._builder._svg_attributes
        )).decode()
    def _repr_svg_(self):  # this is where an SVG object asks to be presented as an image, rather than code for an image
        return str(self)

class SVGBuilder:
    def __init__(self, **attributes):
        self._svg_attributes = dict(version="1.1", xmlns="http://www.w3.org/2000/svg")
        self._svg_attributes.update(attributes)
        self.defs = []
    def __getattr__(self, tag):
        def build(*children, **attributes):
            text = []
            out = SVG(tag, {key.rstrip("_").replace("_", "-"): str(value) for key, value in attributes.items()})
            for child in children:
                if isinstance(child, xml.etree.ElementTree.Element):
                    out.append(child)
                else:
                    text.append(str(child))
            out.text = "".join(text)
            out._builder = self
            return out
        return build
    def __call__(self, points):
        return " ".join(" ".join(map(str, x)) if isinstance(x, tuple) else str(x) for x in points)

<br><br><br>

In [ ]:
svg = SVGBuilder(width=400, height=300)

orbit = svg.g(
    svg.circle(cx=280, cy=150, r=5, fill="black"),
    svg.circle(cx=280, cy=150, r=40, stroke="black", fill="none"),
    svg.circle(cx=320, cy=150, r=10, stroke="black", fill="orange"),
    svg.animateTransform(
        attributeName="transform", type_="rotate", dur="10s",
        from_="360 280 150", to="0 280 150", repeatCount="indefinite",
    ),
)

In [ ]:
orbit

In [ ]:
epicycle = svg.g(
    svg.circle(cx=200, cy=150, r=5, fill="black"),
    svg.circle(cx=200, cy=150, r=80, stroke="black", fill="none"),
    orbit,
    svg.animateTransform(
        attributeName="transform", type_="rotate", dur="20s",
        from_="0 200 150", to="360 200 150", repeatCount="indefinite"
    ),
)

In [ ]:
epicycle

In [ ]:
svg.g(
    svg.ellipse(cx=200, cy=150, rx=120, ry=40, stroke="black", fill="none", stroke_dasharray="3 3"),
    epicycle,
)

<br><br><br>

## What if your students aren't comfortable with Jupyter?

The most common way in which I've seen students stumble is by not recognizing that the order in which they run the cells is important. In a spreadsheet, if you change the value or equation of any cell, all other cells update immediately.

There are ways to make notebooks **reactive**, that is, to update all cells when one cell is changed.
* [marimo](https://marimo.io/) is an alternative to Jupyter with this feature built-in.
* [ipyflow](https://github.com/ipyflow/ipyflow?tab=readme-ov-file#readme) is a kernel that runs in Jupyter to provide this feature.

However, not understanding the significance of evaluation order has deeper consequences for programming in general. If your students are going to be doing much Python coding, you'll want them to understand this. (Also, if any steps are time-consuming, you might not want them to re-run automatically!)

<br><br><br>

This is a mini-exercise that I give students if I think they'll need help:

**Run the following cells in order from step 1 to step 5.**

In [ ]:
# step 4
x = x**2

In [ ]:
# step 2
x = x * 2

In [ ]:
# step 1
x = 1

In [ ]:
# step 5
x -= 20

In [ ]:
# step 3
x += 5

**If you ran them in the right order, the value of `x` will be `29`. Run the next cell.**

In [ ]:
x

**Is it 29?**

<br><br><br>

If they drag the cells into order and evaluate from top to bottom, that's an indication that they understand it.

If they ever get their notebook in a mess, remind them that they can "Restart Kernel".

<img src="img/restart-kernel.png" style="width: 750px; max-width: 75%;">

<br><br><br>

Another cross-check that you can add to a notebook are cells that just spit out some of the symbols they're supposed to have imported.

It's pretty common for them to forget to run the `import` statements, especially after doing "Restart Kernel" without "Run All" or "Run up to Selected Cell".

In [ ]:
display

In [ ]:
HTML

In [ ]:
xml

<br><br><br>

## Keeping your git history tidy

Jupyter notebook files (.ipynb) are JSON and plot outputs are inline PNG.

Try looking at `5-too-much-code.ipynb` as raw text: right click on it and do "Open with" → "Editor".

When you make changes to a notebook and push them to GitHub, GitHub keeps track of which raw text lines have changed, which is often unreadable.

For large Jupyter projects in GitHub, I recommend using [Jupytext](https://jupytext.readthedocs.io/), which represents notebooks either as markdown files with code cells in special back-ticks or as Python files with markdown cells in special comments. It saves no outputs.

Example from [a JupyterBook written with Jupytext](https://github.com/hsf-training/deep-learning-intro-for-hep):
* [a page rendered by JupyterBook](https://hsf-training.github.io/deep-learning-intro-for-hep/03-basic-fitting.html)
* [that page as markdown with code cells in special back-ticks](https://raw.githubusercontent.com/hsf-training/deep-learning-intro-for-hep/refs/heads/main/deep-learning-intro-for-hep/03-basic-fitting.md)
* [example git diff](https://github.com/hsf-training/deep-learning-intro-for-hep/commit/1bf2f2a18325e02d1d6fe1af63abadad5025c8c0)